### Step 1: Load the overlay

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")
from pynq.lib import MicroblazeLibrary

lib = MicroblazeLibrary(base.RPI, ['uart'])
device = lib.uart_open(14,15)

### Step 2: Initialize Webcam and HDMI Out

In [2]:
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

In [3]:
# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

In [ ]:
# initialize camera from OpenCV
import cv2

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(videoIn.isOpened()))

Capture device is open: True


### Step 3: Show input frame on HDMI output

In [ ]:
#import cv2
import numpy as np
import math

cmds=[]
flag=0
state=4
act1=[0x55,0x55,0x05,0x06,0x01,0x01,0x00]
act2=[0x55,0x55,0x05,0x06,0x02,0x01,0x00]
act3=[0x55,0x55,0x05,0x06,0x03,0x01,0x00]
act4=[0x55,0x55,0x05,0x06,0x04,0x01,0x00]
act5=[0x55,0x55,0x05,0x06,0x05,0x01,0x00]
act6=[0x55,0x55,0x05,0x06,0x06,0x01,0x00]
act7=[0x55,0x55,0x05,0x06,0x07,0x01,0x00]
act8=[0x55,0x55,0x05,0x06,0x08,0x01,0x00]

def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i
  
    return num

def led0on():
    base.leds[0].on()
    base.leds[1].off()
    base.leds[2].off()
    base.leds[3].off()

def led1on():
    base.leds[1].on()
    base.leds[0].off()
    base.leds[2].off()
    base.leds[3].off()
    
def led2on():
    base.leds[2].on()
    base.leds[1].off()
    base.leds[0].off()
    base.leds[3].off()
    
def led3on():
    base.leds[3].on()
    base.leds[1].off()
    base.leds[2].off()
    base.leds[0].off()

while 1:
    
    try:
        
    # Capture webcam image

        ret, frame_vga = videoIn.read()

    #Display webcam image via HDMI Out
        if (ret):      
            outframe = hdmi_out.newframe()
            outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
        #hdmi_out.writeframe(outframe)
        else:
            raise RuntimeError("Failed to read from camera.")
        
    #Apply the gesture detection to the input.
        
        frame = frame_vga
        kernel = np.ones((3,3),np.uint8)
    
    #define region of interest
        roi=frame[100:400, 100:400]
        
        
        cv2.rectangle(frame,(100,100),(400,400),(0,255,0),0)    
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        
        
         
    # define range of skin color in HSV
        lower_skin = np.array([0,20,70], dtype=np.uint8)
        upper_skin = np.array([20,255,255], dtype=np.uint8)
        
    #extract skin colur imagw  
        mask = cv2.inRange(hsv, lower_skin, upper_skin)
        
   
        
    #extrapolate the hand to fill dark spots within
        mask = cv2.dilate(mask,kernel,iterations = 4)
        
    #blur the image
        mask = cv2.GaussianBlur(mask,(5,5),100) 
        
        
        
    #find contours
        _,contours,hierarchy= cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #find contour of max area(hand)
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        
    #approx the contour a little
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)
       
        
    #make convex hull around hand
        hull = cv2.convexHull(cnt)
        
        #define area of hull and area of hand
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)
      
    #find the percentage of area not covered by hand in convex hull
        arearatio=((areahull-areacnt)/areacnt)*100
    
     #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(approx, returnPoints=False)
        defects = cv2.convexityDefects(approx, hull)
        
    # l = no. of defects
        l=0
        
    #code for finding no. of defects due to fingers
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0])
            pt= (100,180)
            
            
            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            
            #distance between point and convex hull
            d=(2*ar)/a
            
            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            
        
            # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
            if angle <= 90 and d>30:
                l += 1
                cv2.circle(roi, far, 3, [255,0,0], -1)
            
            #draw lines around hand
            cv2.line(roi,start, end, [0,255,0], 2)
            
            
        l+=1
        
        #print corresponding gestures which are in their ranges
        font = cv2.FONT_HERSHEY_SIMPLEX
        if l==1:
            if areacnt<2000:
                cv2.putText(frame,'Put hand in the box',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            else:
                if arearatio<12:
                    cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('0')
                    flag+=1
                elif arearatio<17.5:
                    cv2.putText(frame,'Best of luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('6')
                    flag+=1
                else:
                    cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('1')
                    flag+=1
        elif l==2:
            cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            cmds.extend('2')
            flag+=1
        elif l==3:
         
                if arearatio<27:
                    cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('3')
                    flag+=1
                else:
                    cv2.putText(frame,'ok',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('7')
                    flag+=1
        elif l==4:
            cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            cmds.extend('4')
            flag+=1
        elif l==5:
            cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            cmds.extend('5')
            flag+=1
        elif l==6:
            cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        else :
            cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        
    # Output OpenCV results via HDMI
        outframe[0:480,0:640,:] = frame[0:480,0:640,:]
        hdmi_out.writeframe(outframe)

    except:
        pass

    if flag==80:
        actcmd=most_frequent(cmds)
        
        #uart stuff
        if(base.switches[1].read()==1):
            
            print('state:',state,'    actcmd:',actcmd)
            
            if state==1:
                led0on()
                if actcmd=='1':
                    lib.uart_write(device,act1, len(act1))
                    print('Pick up 1')
                    state=2
                elif actcmd=='2':
                    lib.uart_write(device,act2, len(act2))
                    print('Pick up 2')
                    state=2
                elif actcmd=='3':
                    lib.uart_write(device,act3, len(act3))
                    print('Pick up 3')
                    state=2
                elif actcmd=='4':
                    lib.uart_write(device,act4, len(act4))
                    print('Pick up 4')
                    state=2
                
            elif state==2:
                led1on()
                if actcmd=='5':
                    state=3
                    
            elif state==3:
                led2on()
                if actcmd=='1':
                    lib.uart_write(device,act5, len(act5))
                    print('Put down 1')
                    state=4
                elif actcmd=='2':
                    lib.uart_write(device,act6, len(act6))
                    print('Put down 2')
                    state=4
                elif actcmd=='3':
                    lib.uart_write(device,act7, len(act7))
                    print('Put down 3')
                    state=4
                elif actcmd=='4':
                    lib.uart_write(device,act8, len(act8))
                    print('Put down 4')
                    state=4
                
                
            elif state==4:
                led3on()
                if actcmd=='7':
                    state=1
                
        cmds.clear()
        flag=0
    
    if(base.buttons[0].read()==1):
        break

In [ ]:
videoIn.release()
hdmi_out.stop()
del hdmi_out